In [9]:
import logging
import os

import torch
import numpy as np
import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist
import pyro.poutine as poutine

from pyro.infer import MCMC, NUTS, Predictive
from pyro.infer.mcmc.util import initialize_model, summary

In [10]:
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.4')

pyro.enable_validation(True)
pyro.set_rng_seed(1)
logging.basicConfig(format='%(message)s', level=logging.INFO)

# Set matplotlib settings
%matplotlib inline
plt.style.use('default')

## Conditional Gaussian Linear Model

In [11]:
# Data
x = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8]).type(torch.Tensor)
y = torch.tensor([5, 10, 16, 25, 26, 32, 34, 40]).type(torch.Tensor)

In [25]:
def model(x, y):
    sigma = pyro.sample("sigma", dist.HalfCauchy(scale=25 * torch.ones(1)))
    beta = pyro.sample("beta", dist.Normal(torch.zeros(1), 5 * torch.ones(1)))
    mu = beta * x
    return pyro.sample("obs", dist.Normal(mu, sigma), obs=y)


In [26]:
nuts_kernel = NUTS(model)
mcmc = MCMC(
    nuts_kernel,
    num_samples=500,
    warmup_steps=300,
)

mcmc.run(x, y) # NOTE: why model and not conditioned model here? 


Sample: 100%|██████████| 800/800 [00:02, 378.58it/s, step size=6.88e-01, acc. prob=0.916]


In [27]:
mcmc.summary(prob=0.5)


                mean       std    median     25.0%     75.0%     n_eff     r_hat
   beta[0]      5.16      0.18      5.16      5.07      5.29    181.35      1.00
  sigma[0]      2.40      0.81      2.24      1.66      2.48    179.75      1.00

Number of divergences: 0
